In [ ]:
# Day 15
#
# Link to task

import math
from copy import deepcopy

from threading import Thread, Lock



def  parse(data):
    return [[int(x) for x in y] for y in data.split("\n")]

def measure(r):
    print("map data is %d %d" % (len(r[0]), len(r)))

def pretty(data, tight=False):
    for l in data:
        o = ""
        for x in l:
            if tight:
                if x is None:
                    o += " "
                else:
                    o += "%d" % (x) 
            else:
                if x is None:
                    o += "    "
                else:
                    o += "% 4d" % (x) 
            
        print(o)

def render(path):
    max_x = 0
    max_y = 0
    for x,y in path:
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
    m = []
    for y in range(max_y+1):
        l = []
        for x in range(max_x+1):
            l.append(None)
        m.append(l)
    
    for i in range(len(path)):
        x,y = path[i]
        m[y][x] = i%10
    return m
        

    
tiny_data = parse("""155
125
111""")
#pretty(tiny_data)

test_data = parse("""1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581""")
#pretty(test_data)

real_data = parse(open("i15.txt").read())
#pretty(real_data)

rw = len(real_data[0])
rh = len(real_data)
hw = rw*5
hh = rh*5
huge_data = []
for y in range(hh):
    l = []
    for x in range(hw):        
        gx = int(x / rw)
        gy = int(y / rh)        
        v = (real_data[y%rh][x%rw] + gx +gy)
        while v > 9:
            v-= 9
        l.append(v)
    huge_data.append(l)

#pretty(huge_data)

print("Data generation done")


def within(w, h, p):
    xx,yy = p
    if xx >= 0 and xx < w:
        if yy >= 0 and yy < h:
            return True
    return False


def processData(collab):
    w = collab["w"]
    h = collab["h"]
    x = None
    y = None
    e_len = 0
    progress = 0
    while True:
        collab["lock"].acquire()
        
        stop = False
        if len(collab["e"]) > 0:
            
            #lowest accumulated risk is sorted to front
            #print("sort before",collab["e"])
            collab["e"].sort(key=lambda x:x[1])
            #print("sort after",collab["e"])
            
            #take out situation
            p, acc = collab["e"].pop(0)
            x,y = p
            
            #write accumulated risk in map
            if collab["n"][y][x] is None:
                collab["n"][y][x] = acc
                #remove other claims on same position from evaluation list
                collab["e"] = list(filter(lambda s: s[0] != p, collab["e"]))
            e_len = len(collab["e"])
            collab["progress"] += 1
            progress = collab["progress"]

            for pp in collab["neighbours"][y][x]:
                xx, yy = pp
                if collab["n"][yy][xx] is None:
                    collab["e"].insert(0, (pp, acc + collab["m"][yy][xx]))
        else:
            stop = True
        collab["lock"].release()
        if stop:
            break
        
        
        if progress % 1000 == 0:
            dxl = x-w
            dyl = y-h
            dist_left = math.sqrt(dxl*dxl+dyl*dyl)
            pct = (progress*1.0/(w*h))*100.0
            print("After %d cycles (%2.2f%%) evaluation list is %d (%d, %d) (%d,%d) %f" % (progress, pct,  e_len, x, y, w, h, dist_left))
        

def flood(m, start):        
    w = len(m[0])
    h = len(m)
    
    if not within(w, h, start):
        print("start has to be within map")
        return None    
    
    #common work
    evaluate = []
    
    #create structire to contain thread locked data
    collab = {}    
    
    #add m and size
    collab["m"] = m
    collab["w"] = w
    collab["h"] = h
    
    nbr = []
    for y in range(h):
        l = []
        for x in range(w):
            nb = []
            for dx,dy in [(1,0),(0,1),(-1,0),(0,-1)]:
                xx = x+dx
                yy = y+dy
                pp = (xx,yy)
                if within(w, h, pp):
                    nb.append(pp)
            l.append(nb)
        nbr.append(l)
    collab["neighbours"] = nbr
    
    
    
    #make flod map
    n = []
    for y in range(h):
        l = []
        for x in range(w):
            l.append(None)
        n.append(l)
    collab["n"] = n
                 
    #seed first situation
    collab["e"] = [(start, 0)]
    collab["progress"] = 0
    collab["lock"] = Lock()
    
    threads = []
    for i in range(4):        
        threads.append(Thread(target = processData, args = (collab,)))
        threads[-1].start()
        print("Started thread %d"%(i))
    print("Work work work")
    for i in range(len(threads)):
        threads[i].join()
        print("Joined thread %d"%(i))
    return collab["n"]

def backtrace(f):
    w = len(f[0])
    h = len(f)
    
    x = w - 1
    y = h - 1
    
    track = [(x,y)]
    
    while True:
        p = track[-1]
        x,y = p
        if p == (0,0):
            #we are home return
            return list(reversed(track))
        best_p = None
        best_v = None
        for dx,dy in [(1,0),(0,1),(-1,0),(0,-1)]:
            xx = x+dx
            yy = y+dy
            pp = (xx,yy)
            if within(w, h, pp):
                if not f[yy][xx] is None:
                    v = f[yy][xx]
                    if best_v == None or v < best_v:
                        best_v = v
                        best_p = pp
        if not best_p is None:
            track.append(best_p)
        
            
    

print("\nTINY")
measure(tiny_data)
pretty(tiny_data, True)
tiny_flood = flood(tiny_data, (0,0))
pretty(tiny_flood)
tiny_bt = backtrace(tiny_flood)
#print(tiny_bt)
pretty(render(tiny_bt), True)
print("Tiny lowest risk is (4)", tiny_flood[-1][-1])

print("\nTEST")
measure(test_data)
pretty(test_data, True)
test_flood = flood(test_data, (0,0))
pretty(test_flood)
test_bt = backtrace(test_flood)
#print(test_bt)
pretty(render(test_bt), True)
print("Test lowest risk is (30)", test_flood[-1][-1])

print("\nREAL")
measure(real_data)
#pretty(real_data, True)
real_flood = flood(real_data, (0,0))
#pretty(real_flood)
real_bt = backtrace(real_flood)
#print(real_bt)
pretty(render(real_bt), True)
print("Real lowest risk is (373)", real_flood[-1][-1])

print("\nHUGE")
measure(huge_data)
#pretty(huge_data, True)
huge_flood = flood(huge_data, (0,0))
#pretty(huge_flood)
huge_bt = backtrace(huge_flood)
#print(huge_bt)
pretty(render(huge_bt), True)
print("Huge lowest risk is (2868)", huge_flood[-1][-1])

print("fin")

Data generation done

TINY
map data is 3 3
155
125
111
Started thread 0
Started thread 1
Started thread 2
Started thread 3
Work work work
Joined thread 0
Joined thread 1
Joined thread 2
Joined thread 3
   0   5  10
   1   3   8
   2   3   4
0  
1  
234
Tiny lowest risk is (4) 4

TEST
map data is 10 10
1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581
Started thread 0
Started thread 1
Started thread 2
Started thread 3
Work work work
Joined thread 0
Joined thread 1
Joined thread 2
Joined thread 3
   0   1   7  10  17  22  23  30  34  36
   1   4  12  11  14  21  23  29  32  34
   3   4   7  13  18  19  20  23  25  33
   6  10  16  17  26  22  21  26  31  38
  13  14  20  20  24  23  28  27  28  29
  14  17  18  27  25  25  33  28  31  36
  15  18  23  32  34  26  28  32  33  34
  18  19  21  26  30  28  29  35  36  43
  19  21  30  29  30  31  37  40  38  39
  21  24  25  26  35  35  39  44  46  40
0         
1         
2345678 

Started thread 0
Started thread 1
Started thread 2
Started thread 3
Work work work
After 1000 cycles (0.40%) evaluation list is 175 (37, 9) (500,500) 674.870358
After 2000 cycles (0.80%) evaluation list is 197 (19, 43) (500,500) 663.483233
After 3000 cycles (1.20%) evaluation list is 337 (63, 3) (500,500) 661.799063
After 4000 cycles (1.60%) evaluation list is 315 (74, 16) (500,500) 644.772828
After 5000 cycles (2.00%) evaluation list is 384 (33, 64) (500,500) 638.893575
After 6000 cycles (2.40%) evaluation list is 369 (73, 50) (500,500) 620.345871
After 7000 cycles (2.80%) evaluation list is 449 (13, 91) (500,500) 635.963835
After 8000 cycles (3.20%) evaluation list is 492 (81, 59) (500,500) 608.310776
After 9000 cycles (3.60%) evaluation list is 497 (49, 89) (500,500) 610.181940
After 10000 cycles (4.00%) evaluation list is 481 (110, 40) (500,500) 603.075451
After 11000 cycles (4.40%) evaluation list is 437 (115, 39) (500,500) 600.621345
After 12000 cycles (4.80%) evaluation list is 

After 100000 cycles (40.00%) evaluation list is 1043 (121, 334) (500,500) 413.759592
After 101000 cycles (40.40%) evaluation list is 1121 (327, 137) (500,500) 402.116898
After 102000 cycles (40.80%) evaluation list is 1053 (347, 127) (500,500) 403.160018
After 103000 cycles (41.20%) evaluation list is 1100 (364, 89) (500,500) 432.916851
After 104000 cycles (41.60%) evaluation list is 1081 (366, 59) (500,500) 460.908885
After 105000 cycles (42.00%) evaluation list is 1109 (181, 309) (500,500) 371.809091
After 106000 cycles (42.40%) evaluation list is 1077 (136, 339) (500,500) 398.016331
After 107000 cycles (42.80%) evaluation list is 1113 (298, 196) (500,500) 364.993151
After 108000 cycles (43.20%) evaluation list is 1151 (377, 26) (500,500) 489.698887
After 109000 cycles (43.60%) evaluation list is 1061 (284, 216) (500,500) 356.808072
After 110000 cycles (44.00%) evaluation list is 1080 (95, 371) (500,500) 425.048233
After 111000 cycles (44.40%) evaluation list is 1125 (165, 332) (500,